In [12]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
tool_wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
import bs4
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vector_db=FAISS.from_documents(documents,OllamaEmbeddings())



C:\Users\sohan\AppData\Local\Temp\ipykernel_24124\125713050.py:9: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  vector_db=FAISS.from_documents(documents,OllamaEmbeddings())


In [7]:
retriever=vector_db.as_retriever()


In [9]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool= create_retriever_tool(retriever,"langsmith-search","Search for information about langsmith. For any question related to langsmith")

In [10]:
retriever_tool.name

'langsmith-search'

In [11]:
## Arxiv wrapper tool
from langchain.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper
api_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=1000)
tool_arxiv=ArxivQueryRun(api_wrapper=api_wrapper)

In [13]:
tools=[tool_wiki,tool_arxiv,retriever_tool]

In [14]:
from langchain_ollama.llms import OllamaLLM
llm=OllamaLLM(model="llama2")

In [65]:
from langchain_core.tools import render_text_description
from langchain.prompts import MessagesPlaceholder
prompt = hub.pull("kryptonite30/structured-chat-agent-prompt")
prompt = prompt.partial(
    
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
    agent_scratchpad="scratchpad",
    chat_history=[],
    
    
)

c:\Users\sohan\Documents\Langchain_Demo\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [66]:
prompt

ChatPromptTemplate(input_variables=['input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.me

In [73]:
##agents
from langchain.agents import create_react_agent,AgentExecutor,create_structured_chat_agent,initialize_agent
agent=create_structured_chat_agent(llm,tools,prompt,)
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True,handle_parsing_errors=True)

In [75]:
agent_executor.invoke({"input":"What is langsmith"})



> Entering new AgentExecutor chain...
Could not parse LLM output: Action: langsmith-search(query: 'langsmith', *, retriever: 'BaseRetriever' = VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002069ED50D70>, search_kwargs={}), document_prompt: 'BasePromptTemplate' = PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator: 'str' = '\n\n', callbacks: 'Callbacks' = None, response_format: "Literal['content', 'content_and_artifact']" = 'content') -> 'Union[str, tuple[str, list[Document]]]'

For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE Invalid or incomplete response

KeyboardInterrupt: 